In [1]:
from get_data import fetch_data # Module to fetch and process data
import pandas as pd
import numpy as np

import folium
from folium.features import CustomIcon
from folium.plugins import HeatMap
import plotly
import plotly.express as px
from dateutil.relativedelta import relativedelta
import datetime
from datetime import date, timedelta

# import config
from sodapy import Socrata
from PIL import Image
import os

In [2]:
api_key = os.environ.get("SOCRATA_APP_TOKEN")
api_key

'ERhyshl2ahv6n185HcntWeML2'

### FETCHING AND PROCESSING DATA

In [5]:
################################
# FETCHING AND PROCESSING DATA #
################################

client = Socrata("data.cityofnewyork.us", api_key)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 100 for testing results, returned as JSON from API / converted to Python list of
# Change to get_all to fetch entire dataset
# dictionaries by sodapy.
# results = client.get_all("h9gi-nx95")
results = client.get("h9gi-nx95", offset=1300000,limit=2000000, order="crash_date")
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


# Cleaning dataset: replacing values identified as erroneous; update as more errors are identified
results_df.replace({
                        'unknown': np.nan,
                        'Unknown': np.nan,
                        '': np.nan, 
                        'Unspecified' : np.nan, 
                        'unspecified' : np.nan, 
                        '80': np.nan, 
                        '1': np.nan, 
                        'Illnes': 'Illness',
                        'Drugs (illegal)': 'Drugs (Illegal)'
                        }, inplace=True)
results_df.drop(['location', 'on_street_name', 'cross_street_name', 'off_street_name', 'vehicle_type_code1', 'vehicle_type_code2', 'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5', 'contributing_factor_vehicle_2', 'contributing_factor_vehicle_3', 'contributing_factor_vehicle_4', 'contributing_factor_vehicle_5'], axis=1, inplace=True)
# collisions_df.dropna(inplace=True) # Dropping all rows with missing values; may revist this later to augment data

# Creating new columns for analysis
results_df['crash_date'] = pd.to_datetime(results_df['crash_date'])
results_df['crash_year'] = pd.DatetimeIndex(results_df['crash_date']).year
results_df['crash_month_year'] = pd.to_datetime(results_df['crash_date']).dt.to_period('M')
results_df['crash_time'] = pd.to_datetime(results_df['crash_time'])
results_df['crash_hour'] = pd.DatetimeIndex(results_df['crash_time']).hour

results_df.number_of_persons_injured.fillna(0, inplace=True)
results_df.number_of_persons_killed.fillna(0, inplace=True)
results_df.number_of_pedestrians_injured.fillna(0, inplace=True)
results_df.number_of_pedestrians_killed.fillna(0, inplace=True)
results_df.number_of_cyclist_injured.fillna(0, inplace=True)
results_df.number_of_cyclist_killed.fillna(0, inplace=True)
results_df.number_of_motorist_injured.fillna(0, inplace=True)
results_df.number_of_motorist_killed.fillna(0, inplace=True)

# Converting objects to int
cols_to_convert = ['number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed']
results_df[cols_to_convert] = results_df[cols_to_convert].astype(int)
results_df = results_df[results_df.crash_date > '2018-12-31']
client.close()

In [6]:
results_df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,collision_id,crash_year,crash_month_year,crash_hour
120406,2019-01-01,2022-01-13 17:00:00,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,4060927,2019,2019-01,17
120407,2019-01-01,2022-01-13 01:30:00,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,Alcohol Involvement,4129434,2019,2019-01,1
120408,2019-01-01,2022-01-13 01:55:00,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,4060684,2019,2019-01,1
120409,2019-01-01,2022-01-13 00:20:00,QUEENS,11377,40.7431370,-73.9158550,0,0,0,0,0,0,0,0,Following Too Closely,4060563,2019,2019-01,0
120410,2019-01-01,2022-01-13 03:30:00,QUEENS,11103,40.7590200,-73.9134500,0,0,0,0,0,0,0,0,Fatigued/Drowsy,4060689,2019,2019-01,3


### KPI CALCULATIONS

In [7]:
results_df.groupby('crash_year')['number_of_cyclist_killed'].sum()

crash_year
2019    0
2020    0
2021    0
2022    0
Name: number_of_cyclist_killed, dtype: int64

In [78]:
collisions = results_df.copy()
last_updated = collisions['crash_date'].max()

grouped_by_day = collisions.groupby(['crash_year', 'crash_date']).agg({
    'collision_id': 'count',
    'number_of_persons_injured': 'sum',
    'number_of_persons_killed': 'sum',
    'number_of_pedestrians_injured': 'sum',
    'number_of_pedestrians_killed': 'sum',
    'number_of_cyclist_injured': 'sum',
    'number_of_cyclist_killed': 'sum',
    'number_of_motorist_injured': 'sum',
    'number_of_motorist_killed': 'sum'
})
grouped_by_day.reset_index(inplace=True)

# Grouping by Year to obtain the cummulative sum
grouped_by_day['collisions_cumsum'] = grouped_by_day.groupby(['crash_year'])['collision_id'].cumsum()

grouped_by_day['person_injured_cumsum'] = grouped_by_day.groupby(['crash_year'])['number_of_persons_injured'].cumsum()
grouped_by_day['person_killed_cumsum'] = grouped_by_day.groupby(['crash_year'])['number_of_persons_killed'].cumsum()

grouped_by_day['peds_injured_cumsum'] = grouped_by_day.groupby(['crash_year'])['number_of_pedestrians_injured'].cumsum()
grouped_by_day['peds_killed_cumsum'] = grouped_by_day.groupby(['crash_year'])['number_of_pedestrians_killed'].cumsum()

grouped_by_day['cyclist_injured_cumsum'] = grouped_by_day.groupby(['crash_year'])['number_of_cyclist_injured'].cumsum()
grouped_by_day['cyclist_killed_cumsum'] = grouped_by_day.groupby(['crash_year'])['number_of_cyclist_killed'].cumsum()
grouped_by_day.reset_index(inplace=True)

# Year to Date metrics
max_date = collisions['crash_date'].max()

collisions_YTD = grouped_by_day.loc[grouped_by_day['crash_date'] == max_date]['collisions_cumsum']

# persons_injured_YTD = grouped_by_month.loc[grouped_by_month['crash_month_year'] == max_date]['person_injured_cumsum']
# persons_killed_YTD = grouped_by_month.loc[grouped_by_month['crash_month_year'] == max_date]['person_killed_cumsum']

peds_injured_YTD = grouped_by_day.loc[grouped_by_day['crash_date'] == max_date]['peds_injured_cumsum']
peds_killed_YTD = grouped_by_day.loc[grouped_by_day['crash_date'] == max_date]['peds_killed_cumsum']

cyclists_injured_YTD = grouped_by_day.loc[grouped_by_day['crash_date'] == max_date]['cyclist_injured_cumsum']
cyclists_killed_YTD = grouped_by_day.loc[grouped_by_day['crash_date'] == max_date]['cyclist_killed_cumsum']

# Previous YTD
last_year = max_date - pd.DateOffset(years=1)
last_YTD= last_year

collisions_YTD_previous = grouped_by_day.loc[grouped_by_day['crash_date'] == last_YTD]['collisions_cumsum']

# persons_injured_YTD_previous = grouped_by_month.loc[grouped_by_month['crash_month_year'] == last_YTD]['person_injured_cumsum']
# persons_killed_YTD_previous = grouped_by_month.loc[grouped_by_month['crash_month_year'] == last_YTD]['person_killed_cumsum']

peds_injured_YTD_previous = grouped_by_day.loc[grouped_by_day['crash_date'] == last_YTD]['peds_injured_cumsum']
peds_killed_YTD_previous = grouped_by_day.loc[grouped_by_day['crash_date'] == last_YTD]['peds_killed_cumsum']

cyclists_injured_YTD_previous = grouped_by_day.loc[grouped_by_day['crash_date'] == last_YTD]['cyclist_injured_cumsum']
cyclists_killed_YTD_previous = grouped_by_day.loc[grouped_by_day['crash_date'] == last_YTD]['cyclist_killed_cumsum']

# Percent Change YTD
collisions_perc_change = (collisions_YTD.values - collisions_YTD_previous.values)/collisions_YTD_previous.values*100

peds_injured_perc_change = (peds_injured_YTD.values - peds_injured_YTD_previous.values)/peds_injured_YTD_previous.values*100
peds_killed_perc_change = (peds_killed_YTD.values - peds_killed_YTD_previous.values)/peds_killed_YTD_previous.values*100

cyclists_injured_perc_change = (cyclists_injured_YTD.values - cyclists_injured_YTD_previous.values)/cyclists_injured_YTD_previous.values*100
cyclists_killed_perc_change = (cyclists_killed_YTD.values - cyclists_killed_YTD_previous.values)/cyclists_killed_YTD_previous.values*100

ipykernel_launcher:63: RuntimeWarning: invalid value encountered in true_divide
ipykernel_launcher:66: RuntimeWarning: invalid value encountered in true_divide


### MAPPING LATEST COLLISIONS

In [93]:
latest_collision_date = results_df.crash_date.max()
latest_collision_date_df = results_df[results_df['crash_date'] == latest_collision_date]
latest_collision_date_df

,crash_date,crash_time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,collision_id,crash_year,crash_month_year,crash_hour
556629,2022-01-08,2022-01-11 01:50:00,NaN,NaN,40.718292,-73.98248,0,0,0,0,0,0,0,0,NaN,4492772,2022,2022-01,1
556630,2022-01-08,2022-01-11 00:55:00,QUEENS,11419,40.69297,-73.81538,0,0,0,0,0,0,0,0,Following Too Closely,4492778,2022,2022-01,0
556631,2022-01-08,2022-01-11 02:25:00,BROOKLYN,11201,40.69195,-73.97897,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,4492785,2022,2022-01,2
556632,2022-01-08,2022-01-11 01:49:00,QUEENS,11372,40.749317,-73.8852,0,0,0,0,0,0,0,0,Backing Unsafely,4492814,2022,2022-01,1
556633,2022-01-08,2022-01-11 04:55:00,QUEENS,11416,40.68815,-73.84884,0,0,0,0,0,0,0,0,Following Too Closely,4492820,2022,2022-01,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556803,2022-01-08,2022-01-11 18:26:00,BROOKLYN,11207,40.65452,-73.882225,0,0,0,0,0,0,0,0,NaN,4493540,2022,2022-01,18
556804,2022-01-08,2022-01-11 21:00:00,BROOKLYN,11208,40.667416,-73.87671,0,0,0,0,0,0,0,0,NaN,4493542,2022,2022-01,21
556805,2022-01-08,2022-01-11 06:00:00,BROOKLYN,11207,40.67576,-73.896645,0,0,0,0,0,0,0,0,Turning Improperly,4493549,2022,2022-01,6
556806,2022-01-08,2022-01-11 14:00:00,BROOKLYN,11216,40.67268,-73.953026,0,0,0,0,0,0,0,0,Backing Unsafely,4493566,2022,2022-01,14


In [213]:
all_dates = []

for d in grouped_by_day.crash_date:
    all_dates.append(d.strftime('%Y-%m-%d'))
all_dates

['2019-01-01',
 '2019-01-02',
 '2019-01-03',
 '2019-01-04',
 '2019-01-05',
 '2019-01-06',
 '2019-01-07',
 '2019-01-08',
 '2019-01-09',
 '2019-01-10',
 '2019-01-11',
 '2019-01-12',
 '2019-01-13',
 '2019-01-14',
 '2019-01-15',
 '2019-01-16',
 '2019-01-17',
 '2019-01-18',
 '2019-01-19',
 '2019-01-20',
 '2019-01-21',
 '2019-01-22',
 '2019-01-23',
 '2019-01-24',
 '2019-01-25',
 '2019-01-26',
 '2019-01-27',
 '2019-01-28',
 '2019-01-29',
 '2019-01-30',
 '2019-01-31',
 '2019-02-01',
 '2019-02-02',
 '2019-02-03',
 '2019-02-04',
 '2019-02-05',
 '2019-02-06',
 '2019-02-07',
 '2019-02-08',
 '2019-02-09',
 '2019-02-10',
 '2019-02-11',
 '2019-02-12',
 '2019-02-13',
 '2019-02-14',
 '2019-02-15',
 '2019-02-16',
 '2019-02-17',
 '2019-02-18',
 '2019-02-19',
 '2019-02-20',
 '2019-02-21',
 '2019-02-22',
 '2019-02-23',
 '2019-02-24',
 '2019-02-25',
 '2019-02-26',
 '2019-02-27',
 '2019-02-28',
 '2019-03-01',
 '2019-03-02',
 '2019-03-03',
 '2019-03-04',
 '2019-03-05',
 '2019-03-06',
 '2019-03-07',
 '2019-03-

In [159]:
s_slider = grouped_by_day.crash_date.max()
e_slider = grouped_by_day.crash_date.min()
delta = e_slider - s_slider   # returns timedelta

for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    print(day)


In [215]:
latest_collision_date_df

,crash_date,crash_time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,collision_id,crash_year,crash_month_year,crash_hour
556630,2022-01-08,2022-01-11 00:55:00,QUEENS,11419,40.69297,-73.81538,0,0,0,0,0,0,0,0,Following Too Closely,4492778,2022,2022-01,0
556631,2022-01-08,2022-01-11 02:25:00,BROOKLYN,11201,40.69195,-73.97897,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,4492785,2022,2022-01,2
556632,2022-01-08,2022-01-11 01:49:00,QUEENS,11372,40.749317,-73.8852,0,0,0,0,0,0,0,0,Backing Unsafely,4492814,2022,2022-01,1
556633,2022-01-08,2022-01-11 04:55:00,QUEENS,11416,40.68815,-73.84884,0,0,0,0,0,0,0,0,Following Too Closely,4492820,2022,2022-01,4
556638,2022-01-08,2022-01-11 11:06:00,MANHATTAN,10022,40.75539,-73.969765,0,0,0,0,0,0,0,0,Oversized Vehicle,4492835,2022,2022-01,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556796,2022-01-08,2022-01-11 04:10:00,QUEENS,11418,40.70695,-73.83785,0,0,0,0,0,0,0,0,Unsafe Speed,4493469,2022,2022-01,4
556798,2022-01-08,2022-01-11 17:22:00,QUEENS,11379,40.71617,-73.882385,0,0,0,0,0,0,0,0,Failure to Yield Right-of-Way,4493489,2022,2022-01,17
556805,2022-01-08,2022-01-11 06:00:00,BROOKLYN,11207,40.67576,-73.896645,0,0,0,0,0,0,0,0,Turning Improperly,4493549,2022,2022-01,6
556806,2022-01-08,2022-01-11 14:00:00,BROOKLYN,11216,40.67268,-73.953026,0,0,0,0,0,0,0,0,Backing Unsafely,4493566,2022,2022-01,14


In [220]:
###########################
# COLLISIONS IN PAST WEEK #
###########################

latest_collision_date = results_df.crash_date.max()
latest_collision_date_df = results_df[results_df['crash_date'] == latest_collision_date].dropna()
locations = zip(latest_collision_date_df.latitude, latest_collision_date_df.longitude)


# Initializing a new map centered around the US
collision_map = folium.Map(location=[40.7128, -74.0060], zoom_start=11, dragging=False, scrollWheelZoom=False,tiles='cartodbpositron')

# Adding maps layers
tooltip = "Click for more info!"
for (_, row) in latest_collision_date_df.iterrows(): # Iterrating through all teams
    
    icon = './Assets/traffic-accident.png'
    icon_image = Image.open(icon)
    
    icon = CustomIcon(
    np.array(icon_image),
    icon_size=(20, 20),
    popup_anchor=(0, -30),
    )
    html = '<i>Cause of Collision: </i>' + '<b>' + row.loc['contributing_factor_vehicle_1'] + '</b>' '<i>Number of Persons Injured: </i>' + '<b>' + str(row.loc['number_of_persons_injured']) + '</b>' + '<b>' + '<i>Number of Persons Killed: </i>' + '<b>' + str(row.loc['number_of_persons_killed']) + '</b>'
    iframe = folium.IFrame(html=html, width=330, height=106)
    popup = folium.Popup(iframe, max_width=330, min_width=330)
    
    folium.Marker(location=[row.loc['latitude'], row.loc['longitude']], icon=icon, popup=popup, tooltip=tooltip).add_to(collision_map)
    
    HeatMap(locations).add_to(collision_map)
locations = zip(latest_collision_date_df.latitude, latest_collision_date_df.longitude)
collision_map

In [172]:
latest_collision_date_df.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,collision_id,crash_year,crash_month_year,crash_hour
556630,2022-01-08,2022-01-11 00:55:00,QUEENS,11419,40.69297,-73.81538,0,0,0,0,0,0,0,0,Following Too Closely,4492778,2022,2022-01,0
556631,2022-01-08,2022-01-11 02:25:00,BROOKLYN,11201,40.69195,-73.97897,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,4492785,2022,2022-01,2
556632,2022-01-08,2022-01-11 01:49:00,QUEENS,11372,40.749317,-73.8852,0,0,0,0,0,0,0,0,Backing Unsafely,4492814,2022,2022-01,1
556633,2022-01-08,2022-01-11 04:55:00,QUEENS,11416,40.68815,-73.84884,0,0,0,0,0,0,0,0,Following Too Closely,4492820,2022,2022-01,4
556638,2022-01-08,2022-01-11 11:06:00,MANHATTAN,10022,40.75539,-73.969765,0,0,0,0,0,0,0,0,Oversized Vehicle,4492835,2022,2022-01,11


In [176]:
# import plotly.graph_objects as go

# import pandas as pd


# fig = go.Figure(data=go.Scattergeo(
#         lon = latest_collision_date_df.longitude, lat = latest_collision_date_df.latitude,
#         # text = latest_collision_date_df,
#         mode = 'markers',
#         # marker_color = latest_collision_date_df['cnt'],
#         ))

# fig.update_layout(
#         title = 'Most trafficked US airports<br>(Hover for airport names)',
#         geo = dict(
#             scope='usa',
#             projection_type='albers usa',
#             showland = True
#         ))

# # focus point
# lat_foc = 35.21401111
# lon_foc = -80.94312583
# fig.update_layout(
#         geo = dict(
#             projection_scale=10, #this is kind of like zoom
#             center=dict(lat=lat_foc, lon=lon_foc), # this will center on the point
#         ))

# fig.show()

In [168]:
latest_collision_date

Timestamp('2022-01-08 00:00:00')

In [65]:
results_df.columns

Index(['crash_date', 'crash_time', 'borough', 'zip_code', 'latitude',
       'longitude', 'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'collision_id', 'crash_year',
       'crash_month_year', 'crash_hour'],
      dtype='object')

In [66]:
results_df.shape

(436402, 19)

In [67]:
min_date = results_df.crash_date.min()
max_date = results_df.crash_date.max()
print(min_date)
print(max_date)

2019-01-01 00:00:00
2022-01-08 00:00:00


ipykernel_launcher:63: RuntimeWarning: invalid value encountered in true_divide
ipykernel_launcher:66: RuntimeWarning: invalid value encountered in true_divide


In [53]:
peds_killed_perc_change

array([nan])

In [50]:
grouped_by_day

,index,crash_year,crash_date,collision_id,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,collisions_cumsum,person_injured_cumsum,person_killed_cumsum,peds_injured_cumsum,peds_killed_cumsum,cyclist_injured_cumsum,cyclist_killed_cumsum
0,0,2018,2018-06-28,567,87,0,2,0,0,0,81,0,567,87,0,2,0,0,0
1,1,2018,2018-06-29,845,98,0,2,0,0,0,94,0,1412,185,0,4,0,0,0
2,2,2018,2018-06-30,651,72,0,4,0,0,0,66,0,2063,257,0,8,0,0,0
3,3,2018,2018-07-01,563,101,0,3,0,0,0,97,0,2626,358,0,11,0,0,0
4,4,2018,2018-07-02,719,94,0,2,0,0,0,90,0,3345,452,0,13,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,1286,2022,2022-01-04,228,41,0,4,0,0,0,37,0,851,172,0,10,0,0,0
1287,1287,2022,2022-01-05,262,51,0,0,0,0,0,51,0,1113,223,0,10,0,0,0
1288,1288,2022,2022-01-06,200,30,0,0,0,0,0,30,0,1313,253,0,10,0,0,0
1289,1289,2022,2022-01-07,236,23,0,0,0,0,0,23,0,1549,276,0,10,0,0,0


In [18]:
grouped_by_month.head(20)

,index,crash_year,crash_month_year,collision_id,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,collisions_cumsum,person_injured_cumsum,person_killed_cumsum,peds_injured_cumsum,peds_killed_cumsum,cyclist_injured_cumsum,cyclist_killed_cumsum
0,0,2012,2012-07,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,2012,2012-08,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0
2,2,2012,2012-09,1,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0
3,3,2012,2012-10,1,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0
4,4,2013,2013-03,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,5,2015,2015-04,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
6,6,2016,2016-03,37,4,0,0,0,0,0,2,0,37,4,0,0,0,0,0
7,7,2016,2016-04,42,8,0,0,0,0,0,8,0,79,12,0,0,0,0,0
8,8,2016,2016-05,7,0,0,0,0,0,0,0,0,86,12,0,0,0,0,0
9,9,2016,2016-06,1,0,0,0,0,0,0,0,0,87,12,0,0,0,0,0


In [19]:
cyclists_killed_perc_change

array([nan])

In [20]:
print(max_date)
print(last_YTD)
print(min_date)

2021-11
2020-11
2012-07-27 00:00:00


In [48]:
cyclists_killed_perc_change

array([], dtype=float64)

In [20]:
results_df.crash_date.max()

Timestamp('2021-04-16 00:00:00')

In [4]:
URL = 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD?$$app_token=' + config.app_token # URL to fetch data in csv format

print(URL)

https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD?$$app_token=ERhyshl2ahv6n185HcntWeML2


In [3]:
collision_df = fetch_data()

ArrowInvalid: CSV parse error: Expected 1 columns, got 2

In [162]:
collisions = collision_df.copy()
collisions['CRASH DATE'] = pd.to_datetime(collisions['CRASH DATE'])
collisions.drop(columns=['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5'], inplace=True)

collisions.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,CRASH YEAR,CRASH MONTH-YEAR,CRASH HOUR
0,2021-04-14,2022-01-10 05:32:00,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,4407480,2021,2021-04,5
1,2021-04-13,2022-01-10 21:35:00,BROOKLYN,11217,40.68358,-73.97617,1.0,0.0,1,0,0,0,0,0,NaN,4407147,2021,2021-04,21
2,2021-04-15,2022-01-10 16:15:00,NaN,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Pavement Slippery,4407665,2021,2021-04,16
3,2021-04-13,2022-01-10 16:00:00,BROOKLYN,11222,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,4407811,2021,2021-04,16
4,2021-04-12,2022-01-10 08:25:00,NaN,NaN,0.00000,0.00000,0.0,0.0,0,0,0,0,0,0,NaN,4406885,2021,2021-04,8


In [230]:
last_updated = collisions['CRASH TIME'].max()

last_updated.strftime("%m/%d/%Y")

'01/10/2022'

In [1]:
grouped_by_month = collisions.groupby(['CRASH YEAR', 'CRASH DATE']).agg({
    'COLLISION_ID': 'count',
    'NUMBER OF PERSONS INJURED': 'sum',
    'NUMBER OF PERSONS KILLED': 'sum',
    'NUMBER OF PEDESTRIANS INJURED': 'sum',
    'NUMBER OF PEDESTRIANS KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum',
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF MOTORIST INJURED': 'sum',
    'NUMBER OF MOTORIST KILLED': 'sum'
})
grouped_by_month.reset_index(inplace=True)

NameError: name 'collisions' is not defined

In [163]:
grouped_by_month = collisions.groupby(['CRASH YEAR', 'CRASH MONTH-YEAR']).agg({
    'COLLISION_ID': 'count',
    'NUMBER OF PERSONS INJURED': 'sum',
    'NUMBER OF PERSONS KILLED': 'sum',
    'NUMBER OF PEDESTRIANS INJURED': 'sum',
    'NUMBER OF PEDESTRIANS KILLED': 'sum',
    'NUMBER OF CYCLIST INJURED': 'sum',
    'NUMBER OF CYCLIST KILLED': 'sum',
    'NUMBER OF MOTORIST INJURED': 'sum',
    'NUMBER OF MOTORIST KILLED': 'sum'
})
grouped_by_month.reset_index(inplace=True)

In [164]:
# grouped_by_month['cyclist_killed_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF CYCLIST KILLED'].cumsum()
# grouped_by_month

In [165]:
grouped_by_month['collisions_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['COLLISION_ID'].cumsum()

grouped_by_month['person_injured_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF PERSONS INJURED'].cumsum()
grouped_by_month['person_killed_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF PERSONS KILLED'].cumsum()

grouped_by_month['peds_injured_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF PEDESTRIANS INJURED'].cumsum()
grouped_by_month['peds_killed_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF PEDESTRIANS KILLED'].cumsum()

grouped_by_month['cyclist_injured_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF CYCLIST INJURED'].cumsum()
grouped_by_month['cyclist_killed_cumsum'] = grouped_by_month.groupby(['CRASH YEAR'])['NUMBER OF CYCLIST KILLED'].cumsum()


grouped_by_month.reset_index(inplace=True)
grouped_by_month.head()

,index,CRASH YEAR,CRASH MONTH-YEAR,COLLISION_ID,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,collisions_cumsum,person_injured_cumsum,person_killed_cumsum,peds_injured_cumsum,peds_killed_cumsum,cyclist_injured_cumsum,cyclist_killed_cumsum
0,0,2012,2012-07,16992,4855.0,22.0,855,6,450,3,3550,13,16992,4855.0,22.0,855,6,450,3
1,1,2012,2012-08,17142,4751.0,27.0,850,15,508,1,3393,11,34134,9606.0,49.0,1705,21,958,4
2,2,2012,2012-09,16535,4689.0,26.0,893,13,434,1,3362,12,50669,14295.0,75.0,2598,34,1392,5
3,3,2012,2012-10,16864,4572.0,21.0,977,13,329,1,3260,7,67533,18867.0,96.0,3575,47,1721,6
4,4,2012,2012-11,15889,3995.0,14.0,1037,8,277,0,2681,6,83422,22862.0,110.0,4612,55,1998,6


In [166]:
max_date = collisions['CRASH MONTH-YEAR'].max()

collisions_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['collisions_cumsum']

persons_injured_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['person_injured_cumsum']
persons_killed_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['person_killed_cumsum']

peds_injured_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['peds_injured_cumsum']
peds_killed_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['peds_killed_cumsum']

cyclists_injured_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['cyclist_injured_cumsum']
cyclists_killed_YTD = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == max_date]['cyclist_killed_cumsum']



In [168]:
last_year = max_date.to_timestamp() - pd.DateOffset(years=1)
last_YTD= last_year.to_period('M')
last_YTD

collisions_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['collisions_cumsum']

persons_injured_YTD_previous = persons_injured_YTD.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['person_injured_cumsum']
persons_killed_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['person_killed_cumsum']

peds_injured_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['peds_injured_cumsum']
peds_killed_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['peds_killed_cumsum']

cyclists_injured_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['cyclist_injured_cumsum']
cyclists_killed_YTD_previous = grouped_by_month.loc[grouped_by_month['CRASH MONTH-YEAR'] == last_YTD]['cyclist_killed_cumsum']

In [194]:
collisions_perc_change = (collisions_YTD.values - collisions_YTD_previous.values)/collisions_YTD_previous.values*100

peds_injured_perc_change = (peds_injured_YTD.values - peds_injured_YTD_previous.values)/peds_injured_YTD_previous.values*100
peds_killed_perc_change = (peds_killed_YTD.values - peds_killed_YTD_previous.values)/peds_killed_YTD_previous.values*100

cyclists_injured_perc_change = (cyclists_injured_YTD.values - cyclists_injured_YTD_previous.values)/cyclists_injured_YTD_previous.values*100
cyclists_killed_perc_change = (cyclists_killed_YTD.values - cyclists_killed_YTD_previous.values)/cyclists_killed_YTD_previous.values*100



In [198]:
grouped_by_month[grouped_by_month['CRASH YEAR'] == 2021]

,index,CRASH YEAR,CRASH MONTH-YEAR,COLLISION_ID,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,collisions_cumsum,person_injured_cumsum,person_killed_cumsum,peds_injured_cumsum,peds_killed_cumsum,cyclist_injured_cumsum,cyclist_killed_cumsum
102,102,2021,2021-01,7689,3112.0,17.0,529,9,284,2,2300,6,7689,3112.0,17.0,529,9,284,2
103,103,2021,2021-02,6962,2549.0,12.0,389,9,142,0,2018,3,14651,5661.0,29.0,918,18,426,2
104,104,2021,2021-03,8258,3526.0,18.0,530,10,324,0,2632,8,22909,9187.0,47.0,1448,28,750,2
105,105,2021,2021-04,8749,4094.0,27.0,541,15,355,0,3030,10,31658,13281.0,74.0,1989,43,1105,2
106,106,2021,2021-05,10281,5035.0,33.0,642,17,511,1,3639,13,41939,18316.0,107.0,2631,60,1616,3
107,107,2021,2021-06,10606,5085.0,27.0,627,6,547,5,3626,15,52545,23401.0,134.0,3258,66,2163,8
108,108,2021,2021-07,9998,4826.0,29.0,569,9,496,3,3507,16,62543,28227.0,163.0,3827,75,2659,11
109,109,2021,2021-08,9872,4909.0,27.0,585,12,532,0,3545,13,72415,33136.0,190.0,4412,87,3191,11
110,110,2021,2021-09,9883,4968.0,23.0,713,7,498,3,3480,13,82298,38104.0,213.0,5125,94,3689,14
111,111,2021,2021-10,10192,5027.0,33.0,805,15,543,3,3417,12,92490,43131.0,246.0,5930,109,4232,17


In [ ]:
grouped_by_month[grouped_by_month['CRASH YEAR'] == 2021]

In [44]:
########
# KPIs #
########

max_date = collisions['CRASH MONTH-YEAR'].max()

grouped_by_month_cumsum = grouped_by_month.cumsum(axis=0).reset_index()
grouped_by_month_cumsum




,CRASH MONTH-YEAR,COLLISION_ID,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED
0,2012-07,16992,4855.0,22.0,855,6,450,3,3550,13
1,2012-08,34134,9606.0,49.0,1705,21,958,4,6943,24
2,2012-09,50669,14295.0,75.0,2598,34,1392,5,10305,36
3,2012-10,67533,18867.0,96.0,3575,47,1721,6,13565,43
4,2012-11,83422,22862.0,110.0,4612,55,1998,6,16246,49
...,...,...,...,...,...,...,...,...,...,...
110,2021-09,1827084,512169.0,2398.0,94760,1240,43406,181,372204,967
111,2021-10,1837276,517196.0,2431.0,95565,1255,43949,184,375621,979
112,2021-11,1846622,521692.0,2455.0,96329,1266,44341,185,378773,991
113,2021-12,1855060,525748.0,2473.0,97121,1272,44664,186,381551,1001


In [123]:
persons_killed_YTD = grouped_by_month_cumsum.loc[grouped_by_month_cumsum['CRASH MONTH-YEAR'] == max_date]['NUMBER OF PERSONS KILLED']
persons_killed_YTD

114    2477.0
Name: NUMBER OF PERSONS KILLED, dtype: float64

In [35]:
print(f'Data through {max_date}')

Data through 2022-01


In [31]:
collisions['CRASH TIME'] = pd.to_datetime(collisions['CRASH TIME'])
collisions['CRASH HOUR'] = collisions['CRASH TIME'].dt.floor('H')

In [32]:
collisions.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,...,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,CRASH YEAR,CRASH MONTH-YEAR,CRASH HOUR,HOUR
0,04/14/2021,2022-01-10 05:32:00,NaN,NaN,NaN,NaN,0.0,0.0,0,0,...,NaN,Sedan,Sedan,NaN,NaN,NaN,2021,2021-04,2022-01-10 05:00:00,6
1,04/13/2021,2022-01-10 21:35:00,BROOKLYN,11217,40.68358,-73.97617,1.0,0.0,1,0,...,NaN,Sedan,NaN,NaN,NaN,NaN,2021,2021-04,2022-01-10 21:00:00,22
2,04/15/2021,2022-01-10 16:15:00,NaN,NaN,NaN,NaN,0.0,0.0,0,0,...,NaN,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,2021,2021-04,2022-01-10 16:00:00,16
3,04/13/2021,2022-01-10 16:00:00,BROOKLYN,11222,NaN,NaN,0.0,0.0,0,0,...,NaN,Sedan,NaN,NaN,NaN,NaN,2021,2021-04,2022-01-10 16:00:00,16
4,04/12/2021,2022-01-10 08:25:00,NaN,NaN,0.00000,0.00000,0.0,0.0,0,0,...,NaN,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,2021,2021-04,2022-01-10 08:00:00,8


In [50]:
collisions.fillna({'CONTRIBUTING FACTOR VEHICLE 1':1,'CONTRIBUTING FACTOR VEHICLE 2':1, 'CONTRIBUTING FACTOR VEHICLE 3':1, 'CONTRIBUTING FACTOR VEHICLE 4':1, 'CONTRIBUTING FACTOR VEHICLE 5':1}, inplace=True)

collisions['MULTI-CAR'] =  collisions[['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5']].sum(numeric_only=True)



In [51]:
collisions.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,MULTI-CAR
0,04/14/2021,5:32,NaN,NaN,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,NaN,NaN,...,1,1,1,4407480,Sedan,Sedan,NaN,NaN,NaN,NaN
1,04/13/2021,21:35,BROOKLYN,11217,40.68358,-73.97617,"(40.68358, -73.97617)",NaN,NaN,620 ATLANTIC AVENUE,...,1,1,1,4407147,Sedan,NaN,NaN,NaN,NaN,NaN
2,04/15/2021,16:15,NaN,NaN,NaN,NaN,NaN,HUTCHINSON RIVER PARKWAY,NaN,NaN,...,1,1,1,4407665,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN
3,04/13/2021,16:00,BROOKLYN,11222,NaN,NaN,NaN,VANDERVORT AVENUE,ANTHONY STREET,NaN,...,1,1,1,4407811,Sedan,NaN,NaN,NaN,NaN,NaN
4,04/12/2021,8:25,NaN,NaN,0.00000,0.00000,"(0.0, 0.0)",EDSON AVENUE,NaN,NaN,...,1,1,1,4406885,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,NaN


In [52]:
collisions.drop(['LOCATION', 'COLLISION_ID', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME'], axis=1, inplace=True)
collisions['CRASH YEAR'] = pd.DatetimeIndex(collisions['CRASH DATE']).year
collisions['CRASH MONTH-YEAR'] = pd.to_datetime(collisions['CRASH DATE']).dt.to_period('M')
collisions.head()


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,MULTI-CAR,CRASH YEAR,CRASH MONTH-YEAR
0,04/14/2021,5:32,NaN,NaN,NaN,NaN,0.0,0.0,0,0,...,1,1,Sedan,Sedan,NaN,NaN,NaN,NaN,2021,2021-04
1,04/13/2021,21:35,BROOKLYN,11217,40.68358,-73.97617,1.0,0.0,1,0,...,1,1,Sedan,NaN,NaN,NaN,NaN,NaN,2021,2021-04
2,04/15/2021,16:15,NaN,NaN,NaN,NaN,0.0,0.0,0,0,...,1,1,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,2021,2021-04
3,04/13/2021,16:00,BROOKLYN,11222,NaN,NaN,0.0,0.0,0,0,...,1,1,Sedan,NaN,NaN,NaN,NaN,NaN,2021,2021-04
4,04/12/2021,8:25,NaN,NaN,0.00000,0.00000,0.0,0.0,0,0,...,1,1,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,NaN,2021,2021-04


In [53]:
list(collisions['VEHICLE TYPE CODE 1'].unique())

['Sedan',
 'Station Wagon/Sport Utility Vehicle',
 'Taxi',
 '�MBU',
 'Pick-up Truck',
 'Box Truck',
 'Bus',
 nan,
 'Ambulance',
 'Tow Truck / Wrecker',
 'E-Bike',
 'Van',
 'Flat Bed',
 'Moped',
 'Tractor Truck Diesel',
 'AMBULANCE',
 'Bike',
 'Trailer',
 'Motorcycle',
 'Garbage or Refuse',
 'SCHOOL BUS',
 'Lift Boom',
 'scooter',
 'CATER',
 'PKUP',
 'Carry All',
 'FDNY Ambul',
 '3-Door',
 'Beverage Truck',
 'SEMI TRAIL',
 'Dump',
 'Convertible',
 '4 dr sedan',
 'dump',
 'Flat Rack',
 'GARBAGE TR',
 'Motorbike',
 'Tanker',
 'Armored Truck',
 'MOPED',
 'SW/VAN',
 'Bulk Agriculture',
 'Motorscooter',
 'UTILITY',
 'Concrete Mixer',
 'Open Body',
 'E-Scooter',
 'unknown',
 'fire truck',
 'Glass Rack',
 'COMMERCIAL',
 'PSD',
 'Scooter',
 'MTA',
 'Multi-Wheeled Vehicle',
 'PK',
 'Minicycle',
 'TRUCK',
 'Truck',
 'Chassis Cab',
 'LIMO',
 'Tractor Truck Gasoline',
 'MACK',
 'Refrigerated Van',
 'van',
 'CARRIER',
 'Fire truck',
 'ambulance',
 'NYPD VAN',
 '2 dr sedan',
 'MTR H',
 'Pallet',
 'AM

In [54]:
collisions['VEHICLE TYPE CODE 2'] = collisions['VEHICLE TYPE CODE 1'].str.title()
collisions['VEHICLE TYPE CODE 2'].replace({'scoo': 'Scooter', '11111': np.nan}, inplace=True)
collisions['VEHICLE TYPE CODE 2'].unique()


array(['Sedan', 'Station Wagon/Sport Utility Vehicle', 'Taxi', '�Mbu',
       'Pick-Up Truck', 'Box Truck', 'Bus', nan, 'Ambulance',
       'Tow Truck / Wrecker', 'E-Bike', 'Van', 'Flat Bed', 'Moped',
       'Tractor Truck Diesel', 'Bike', 'Trailer', 'Motorcycle',
       'Garbage Or Refuse', 'School Bus', 'Lift Boom', 'Scooter', 'Cater',
       'Pkup', 'Carry All', 'Fdny Ambul', '3-Door', 'Beverage Truck',
       'Semi Trail', 'Dump', 'Convertible', '4 Dr Sedan', 'Flat Rack',
       'Garbage Tr', 'Motorbike', 'Tanker', 'Armored Truck', 'Sw/Van',
       'Bulk Agriculture', 'Motorscooter', 'Utility', 'Concrete Mixer',
       'Open Body', 'E-Scooter', 'Unknown', 'Fire Truck', 'Glass Rack',
       'Commercial', 'Psd', 'Mta', 'Multi-Wheeled Vehicle', 'Pk',
       'Minicycle', 'Truck', 'Chassis Cab', 'Limo',
       'Tractor Truck Gasoline', 'Mack', 'Refrigerated Van', 'Carrier',
       'Nypd Van', '2 Dr Sedan', 'Mtr H', 'Pallet', 'Ambulane', 'Amb',
       'Mailtruck', 'Com', 'Util', 'Pick Up

In [55]:
collisions.groupby('CRASH YEAR').sum()

,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,MULTI-CAR
CRASH YEAR,,,,,,,,,,,
2012,3.479906e+06,-6.317853e+06,27453.0,137.0,5906,72,2210,6,19331,59,0.0
2013,7.001534e+06,-1.271036e+07,55124.0,297.0,11988,176,4075,11,39060,110,0.0
2014,7.034421e+06,-1.276935e+07,51223.0,262.0,11036,133,4000,20,36176,109,0.0
2015,7.451619e+06,-1.352611e+07,51358.0,243.0,10084,133,4281,15,36992,95,0.0
2016,7.826467e+06,-1.420810e+07,60317.0,246.0,11090,149,4975,18,44010,72,0.0
2017,8.836971e+06,-1.604679e+07,60657.0,256.0,11151,127,4889,27,44616,107,0.0
2018,8.802428e+06,-1.598224e+07,61939.0,231.0,11123,123,4725,10,46068,98,0.0
2019,7.893567e+06,-1.432579e+07,61389.0,244.0,10568,131,4986,31,45834,82,0.0
2020,4.227987e+06,-7.673009e+06,44605.0,269.0,6689,102,5576,29,32340,138,0.0


In [56]:
cols_for_kepler = ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED']

In [57]:
kepler_df = collision_df[cols_for_kepler].dropna()

In [58]:
kepler_df.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED
1,04/13/2021,21:35,BROOKLYN,11217,40.683580,-73.97617,1.0,0.0,1,0,0,0,0,0
13,05/21/2019,22:50,BROOKLYN,11201,40.697540,-73.98312,0.0,0.0,0,0,0,0,0,0
15,02/26/2021,14:50,BRONX,10461,40.843464,-73.83600,0.0,0.0,0,0,0,0,0,0
17,03/31/2021,22:20,BROOKLYN,11234,40.626457,-73.91800,1.0,0.0,0,0,0,0,1,0
18,04/06/2021,22:58,STATEN ISLAND,10312,40.526894,-74.16728,7.0,0.0,0,0,0,0,7,0


In [23]:
for row in collision_df.values:
    print(row)

In [17]:
# Load a map with data and config and height
from keplergl import KeplerGl
map_2 = KeplerGl()
map_2.add_data(data=kepler_df, name='Summary of Vehile Collisions in NYC')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [ ]:
map_2